In [2]:
from dataclasses import dataclass, field
from math import copysign
from typing import Dict, List, Tuple

from pyprojroot import here

In [59]:
@dataclass
class HeadKnot:
    x = 0
    y = 0

    def move(self, direction: str):
        if direction == 'L':
            self.x -= 1

        elif direction == 'U':
            self.y += 1

        elif direction == 'R':
            self.x += 1

        elif direction == 'D':
            self.y -= 1


@dataclass
class TailKnot(HeadKnot):
    headKnot: HeadKnot

    def move(self):
        relativeX = self.headKnot.x - self.x
        relativeY = self.headKnot.y - self.y
        touching = abs(relativeX) < 2 and abs(relativeY) < 2

        if not touching:
            if abs(relativeX) > 0:
                self.x += int(copysign(1, relativeX))
                    
            if abs(relativeY) > 0:
                self.y += int(copysign(1, relativeY))


@dataclass
class Rope:
    numKnots: int
    headKnot: HeadKnot = field(init=False)
    tailKnots: List[TailKnot] = field(init=False)
    tailPositions: set = field(default_factory=set)

    def __post_init__(self):
        self.headKnot = HeadKnot()
        self.tailKnots = []

        headKnot = self.headKnot
        for _ in range(self.numKnots - 1):
            tailKnot = TailKnot(headKnot)
            self.tailKnots.append(tailKnot)
            headKnot = tailKnot

        self.tailPositions.add((0,0))

    def move(self, direction: str, numSteps: int):
        for step in range(numSteps):
            self.headKnot.move(direction)

            for tailKnot in self.tailKnots:
                tailKnot.move()
            
            self.tailPositions.add((tailKnot.x, tailKnot.y)) # type: ignore

In [60]:
def ropeBridge(moves) -> int:
    rope = Rope(10)
    
    for direction, numSteps in moves:
        rope.move(direction, numSteps)
    
    return len(rope.tailPositions)

In [64]:
path = here('./09/input-1.txt')
with open(path) as fp:
    moves = [(direction, int(steps)) for direction, steps in [tuple(line.strip().split(' ')) for line in fp.readlines()]]

numTailPositions = ropeBridge(moves)
numTailPositions

2303